In [73]:
import pandas as pd
from datetime import datetime
from datetime import date
import json
import math
import numpy as np

with open('lead_times_dict.json', 'r') as file:
    lead_times_dict = json.load(file)
    
with open('S_factor_dict.json', 'r') as file:
    S_factor_dict = json.load(file)

def stats(dataframe, confianza):

    mean_no_group = dataframe.groupby(by=['ID_SKU_VENTA', 'DESCR_CENDIST']).mean().reset_index().rename(columns={'Venta en pallets': 'MEDIA'})
    stdev_no_group = dataframe.groupby(by=['ID_SKU_VENTA', 'DESCR_CENDIST']).std().reset_index().rename(columns={'Venta en pallets': 'STD'})
    
    mean_no_group['MEDIA'] = mean_no_group.apply(lambda x: x.MEDIA * lead_times_dict[x['DESCR_CENDIST']], axis=1)
    stdev_no_group['STD'] = stdev_no_group.apply(lambda x: x.STD * lead_times_dict[x['DESCR_CENDIST']], axis=1)

    data_completa = mean_no_group.merge(stdev_no_group, on=['ID_SKU_VENTA', 'DESCR_CENDIST'])

    data_completa['s'] = data_completa.apply(lambda x:  x['MEDIA'] + (confianza * x['STD']), axis=1)
    
    data_completa['S'] = data_completa.apply(lambda x: x['s'] * int(S_factor_dict[str(x['ID_SKU_VENTA'])]), axis=1)
    
    data_completa['INVENTARIO_PROMEDIO'] = data_completa.apply(lambda x: (x['s'] + x['S']) / 2, axis=1)
    
    return data_completa.drop(columns=['MEDIA', 'STD', 'Unnamed: 0_x', 'Unnamed: 0_y', 's', 'S'])

datos = pd.read_csv('../../data/datos_limpios_filtrados.csv')
print(datos['FECHA'].drop_duplicates().size)

zone_dict = {'ARICA': 'NORTE GRANDE',
 'IQUIQUE': 'NORTE GRANDE',
 'ANTOFAGASTA': 'NORTE GRANDE',
 'COPIAPO': 'NORTE CHICO',
 'COQUIMBO': 'NORTE CHICO',
 'OVALLE': 'NORTE CHICO',
 'ILLAPEL': 'NORTE CHICO',
 'LLAY LLAY': 'CENTRO',
 'CURAUMA': 'CENTRO',
 'SANTIAGO SUR': 'CENTRO',
 'RANCAGUA': 'CENTRO',
 'TALCA': 'CENTRO SUR',
 'CHILLAN': 'CENTRO SUR',
 'TALCAHUANO': 'CENTRO SUR',
 'LOS ANGELES': 'CENTRO SUR',
 'TEMUCO': 'SUR',
 'VALDIVIA': 'SUR',
 'OSORNO': 'SUR',
 'PUERTO MONTT': 'SUR',
 'CASTRO': 'SUR',
 'COYHAIQUE': 'SUR',
 'CALAMA': 'NORTE GRANDE'}

832


In [74]:
foto = stats(datos, 1.65)
foto = foto.drop(columns=['Unnamed: 0.1_x', 'Unnamed: 0.1_y'])

foto['ZONA'] = foto.apply(lambda x: zone_dict[x['DESCR_CENDIST']], axis=1)

In [75]:
foto.groupby(by=['ZONA']).sum().drop(columns=['ID_SKU_VENTA'])

,INVENTARIO_PROMEDIO
ZONA,
CENTRO,10680.251921
CENTRO SUR,11517.105601
NORTE CHICO,4794.623541
NORTE GRANDE,13304.806504
SUR,10676.129434


In [63]:
foto.groupby(by=['DESCR_CENDIST']).sum().drop(columns=['ID_SKU_VENTA'])

,INVENTARIO_PROMEDIO
DESCR_CENDIST,
ANTOFAGASTA,3052.266759
ARICA,4702.528167
CALAMA,1934.017191
CASTRO,822.487790
CHILLAN,1405.437839
COPIAPO,1894.403808
COQUIMBO,2032.744189
COYHAIQUE,1995.001495
CURAUMA,5184.682337
